In [1]:
!pip install -q bitsandbytes accelerate transformers peft datasets huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from huggingface_hub import login
login()


In [23]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_id = "mistralai/Mistral-7B-v0.1"


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token


model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True
)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [29]:
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split


df = pd.read_csv("/content/dataset_fp.csv")
df.columns = df.columns.str.strip()
df = df.rename(columns={"Research Question": "problem", "Approach": "approach"})


train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)


dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "test": Dataset.from_pandas(test_df)
})



In [24]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training


def tokenize(example):
    return tokenizer(example["problem"], padding="max_length", truncation=True, max_length=512)


tokenized_dataset = dataset.map(tokenize, batched=True)


tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])


model = prepare_model_for_kbit_training(model)


lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)


model = get_peft_model(model, lora_config)



Map:   0%|          | 0/92 [00:00<?, ? examples/s]

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

In [17]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling


training_args = TrainingArguments(
    output_dir="./mistral-finetuned",
    per_device_train_batch_size=1,
    num_train_epochs=3,
    logging_steps=1,
    save_strategy="no",
    report_to="none",
    fp16=True,
    gradient_accumulation_steps=4,
)


data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator
)


trainer.train()


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
1,2.405200
2,2.356300
3,2.407200
4,2.421500
5,2.743500
6,2.403800
7,2.307200
8,2.336300
9,2.198500
10,2.205400


TrainOutput(global_step=69, training_loss=1.9767169451368027, metrics={'train_runtime': 708.9734, 'train_samples_per_second': 0.389, 'train_steps_per_second': 0.097, 'total_flos': 6031819036164096.0, 'train_loss': 1.9767169451368027, 'epoch': 3.0})

In [18]:
metrics = trainer.evaluate()
print(metrics)

{'eval_loss': 1.9172377586364746, 'eval_runtime': 5.6801, 'eval_samples_per_second': 1.937, 'eval_steps_per_second': 0.352, 'epoch': 3.0}


In [30]:
!pip install evaluate rouge_score bert-score

from evaluate import load
from tqdm import tqdm
from bert_score import score

rouge = load("rouge")

preds = []
refs = []

for example in dataset["test"]:
    prompt = f"Research problem: {example['problem']}\nApproach:"
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        output_ids = model.generate(
            **input_ids,
            max_new_tokens=300,
            do_sample=False,
            temperature=0.7
        )


    generated = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    generated_approach = generated.split("Approach:")[-1].strip()

    preds.append(generated_approach)
    refs.append(example["approach"].strip())


rouge_results = rouge.compute(predictions=preds, references=refs)
print("ROUGE scores:", rouge_results)


P, R, F1 = score(preds, refs, lang="en", model_type="bert-base-uncased")
print(f"BERTScore Precision: {P.mean().item():.4f}")
print(f"BERTScore Recall:    {R.mean().item():.4f}")
print(f"BERTScore F1:        {F1.mean().item():.4f}")


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad

ROUGE scores: {'rouge1': np.float64(0.1897828406735395), 'rouge2': np.float64(0.02881728847302749), 'rougeL': np.float64(0.13072402502863747), 'rougeLsum': np.float64(0.1313414849904564)}
BERTScore Precision: 0.5744
BERTScore Recall:    0.5361
BERTScore F1:        0.5538
